<a href="https://colab.research.google.com/github/johnnathan862/pos-tech-datathon/blob/main/creating_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [261]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools

In [126]:
dt_hoje = '2024-08-08'

In [127]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Funções

In [128]:
def drop_unnamed_cols(df):
    cols = df.columns
    cols_unnamed = [col for col in cols if str(col).startswith('Unnamed')]

    columns_to_drop = df[cols_unnamed].dropna(axis=1, how='all').columns

    return df.drop(columns=columns_to_drop)



def drop_null_cols(df):
    return df.dropna(axis=1, how='all')



def read_all_tables(dirname):
    all_dfs = {}

    list_files = os.listdir(dirname)

    for file_ in list_files:
        if file_.endswith(".csv"):
            file_name = os.path.splitext(file_)[0]
            df = pd.read_csv(os.path.join(dirname, file_))
            df_cleanned = drop_unnamed_cols(df)
            df_cleanned = drop_null_cols(df_cleanned)
            all_dfs[file_name] = df_cleanned

            print(file_)

    return all_dfs



def print_keys_with_column(data_dict, column_name):
    for key, df in data_dict.items():
        if column_name in df.columns:
            print(key)



def print_keys_with_column_like(data_dict, column_name_like):
    for key, df in data_dict.items():
        matching_columns = [col for col in df.columns if column_name_like in col]
        if matching_columns:
            print(key)



def search_tables_use_column(ls_data_dict, type_search, column_name):
    if type_search == 'like':
        for dict_data in ls_data_dict:
            print_keys_with_column_like(dict_data, column_name)
    elif type_search == 'exact':
        for dict_data in ls_data_dict:
            print_keys_with_column(dict_data, column_name)
    else:
        raise ValueError("Tipo de busca não suportado. Use 'like' ou 'exact'.")



def plot_qtd_stack_bar(df, x, y, group, figsize=(20,10), colors=None):
    # Se a group for nulo o gráfico de barras pode ver utilizado sem criar grupos dentro das barras
    if group:
        agg_tips = df.groupby([x, group])[y].count().unstack().fillna(0)
    else:
        agg_tips = df.groupby([x])[y].count().fillna(0).to_frame()

    fig, ax = plt.subplots(figsize=figsize)

    # colors = ['#008080', '#DAA520','#8A2BE2','#0000FF']
    if colors:
        pass
    else:
        colors = [None for i in range(agg_tips.shape[1])]

    bottom = np.zeros(len(agg_tips))

    lst_index = [str(i) for i in agg_tips.index]

    for i, col in enumerate(agg_tips.columns):
        ax.bar(
          lst_index, agg_tips[col], bottom=bottom, label=col, edgecolor = "black", color=colors[i]) #, color=colors[i]
        bottom += np.array(agg_tips[col])

    # Coloca o label de qtd de observações acima das barras.
    totals = agg_tips.sum(axis=1)
    lst_index_total = [str(i) for i in totals.index]
    y_offset = 5
    for i, total in enumerate(totals):
        ax.text(lst_index_total[i], total + y_offset, round(total), ha='center', weight='bold')

    # Se group é nulo não colocamos o label de qtd de observações dentro da barra
    if group:
        # Let's put the annotations inside the bars themselves by using a
        # negative offset.
        y_offset = -100
        # For each patch (basically each rectangle within the bar), add a label.
        for bar in ax.patches:
            ax.text(
                # Put the text in the middle of each bar. get_x returns the start
                # so we add half the width to get to the middle.
                bar.get_x() + bar.get_width() / 2,
                # Vertically, add the height of the bar to the start of the bar,
                # along with the offset.
                bar.get_y() + bar.get_height() / 2, #bar.get_height() + bar.get_y() + y_offset,
                # This is actual value we'll show.
                round(bar.get_height()),
                # Center the labels and style them a bit.
                ha='center',
                color='w',
                weight='bold',
                size=8
            )

        ax.set_title(f'Qtd de {y} por {group}')
        ax.legend()
        plt.xticks(rotation=90)

    else:
        ax.set_title(f'Qtd de {y}')
        ax.legend()
        plt.xticks(rotation=90)



def plot_porcentagem_stack_bar(df, x, y, group, figsize=(20,10)):
    agg_tips = df.groupby([x, group])[y].count().unstack().fillna(0)
    totals = agg_tips.sum(axis=1)
    agg_tips = agg_tips.div(totals, axis='index').mul(100)

    fig, ax = plt.subplots(figsize=figsize)

    # colors = ['#008080', '#DAA520','#8A2BE2','#0000FF']
    bottom = np.zeros(len(agg_tips))

    lst_index = [str(i) for i in agg_tips.index]

    for i, col in enumerate(agg_tips.columns):
        ax.bar(
          lst_index, agg_tips[col], bottom=bottom, label=col, edgecolor = "black") # , color=colors[i]

        bottom += np.array(agg_tips[col])

    # For each patch (basically each rectangle within the bar), add a label.
    for bar in ax.patches:
        ax.text(
            # Put the text in the middle of each bar. get_x returns the start
            # so we add half the width to get to the middle.
            bar.get_x() + bar.get_width() / 2,
            # Vertically, add the height of the bar to the start of the bar,
            # along with the offset.
            bar.get_y() + bar.get_height() / 2, #bar.get_height() + bar.get_y() + y_offset,
            # This is actual value we'll show.
            round(bar.get_height()),
            # Center the labels and style them a bit.
            ha='center',
            color='w',
            weight='bold',
            size=8
        )

    ax.set_title(f'Porcentagem de matriculas por {group}')
    ax.legend()
    plt.xticks(rotation=90)


def cleanning_cols_dates(df):
    df_ = df.copy()

    lst_cols_dates = [col for col in df_.columns if col.startswith('Data')]
    print("Lista de colunas de datas:", lst_cols_dates)
    for col in lst_cols_dates:
        try:
            df_[col] = pd.to_datetime(df_[col])
        except:
            print(f'Essa coluna "{col}" não pode ser convertida para data')

    return df_


def coalesce_multiple_columns(df, lst_cols_to_coalesce, new_col, drop_cols=False):
    df_ = df.copy()
    # Nova coluna
    df_[new_col] = df_[lst_cols_to_coalesce].bfill(axis=1).iloc[:, 0]

    if drop_cols:
        df_ = df_.drop(columns=lst_cols_to_coalesce)

    return df_

# Lendo todas as tabelas

In [129]:
TbOutras = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/Outras tabelas/")
TbAbatimento = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbAbatimento/Originais anonimizados")
TbAlunos = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbAluno/Originais anonimizados")
TbCampoDinamico = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbCampoDinamico/Originais anonimizados")
TbCaptacao = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbCaptacao/Originais anonimizados")
TbDiario = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbDiario/Originais anonimizados")
TbFase = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbFase/Originais anonimizados")
TbHistorico = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbHistorico/Originais anonimizados")
TbMeta = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbMeta")
TbProfessor = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbProfessor/Originais anonimizados")
TbResponsavel = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbResponsavel/Originais anonimizados")
TbSerie = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbSerie/Originais anonimizados")
TbSituacaoAlunoDisciplina = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbSituacaoAlunoDisciplina/Originais anonimizados")
TbTurma = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbTurma/Originais anonimizados")
lst_todos_dados = [TbOutras, TbAbatimento, TbAlunos, TbCampoDinamico, TbCaptacao, TbDiario, TbFase, TbHistorico, TbMeta, TbProfessor, TbResponsavel, TbSerie, TbSituacaoAlunoDisciplina, TbTurma]

DictTodasTabelas = (TbOutras | TbAbatimento | TbAlunos | TbCampoDinamico | TbCaptacao | TbDiario | TbFase | TbHistorico | TbMeta | TbProfessor | TbResponsavel | TbSerie | TbSituacaoAlunoDisciplina | TbTurma)

TbPais.csv
TbMunicipio.csv
TbDisciplina.csv
TbCentroResultado.csv
TbPeriodo.csv
TbMotivoInativacao.csv
TbFormaIngresso.csv
TbFreqQuadroHorario.csv
TbCursoFases.csv
TbTipoOcorrencia.csv
TbGradeCurricular.csv
TbAbatimentoTipo.csv
TbAbatimento.csv
TbAlunoRotinaEducacaoInfantil.csv
TbAlunoTurmaHistorico.csv
TbAlunoProprioResponsavel.csv
TbAlunoObs.csv
TbAluno.csv
TbAlunoTurma.csv
TbCampoDinamicoConjuntoElemento.csv
TbCampoDinamico.csv
TbCampoDinamicoConjunto.csv
TbCaptacaoCursoInteresse.csv
TbCaptacaoMotivoDesistencia.csv
TbCaptacaoSituacaoLead.csv
TbCaptacaoOrigemLead.csv
TbDiarioFrequencia.csv
TbDiarioAluno.csv
TbDiario.csv
TbDiarioAula.csv
TbFaseNotaAluno.csv
TbFaseNota.csv
TbFaseNotaDisciplinaTurma.csv
TbFaseNotaOrigemDestino.csv
TbHistorico.csv
TbHistoricoNotas.csv
TbMeta.csv
TbMetaFaseNota.csv
TbMetaFaseNotaAluno.csv
TbMetaSituacaoAlunoDisciplina.csv
TbTipoMeta.csv
TbMetaConceito.csv
TbProfessorDisciplina.csv
TbProfessorHorario.csv
TbProfessor.csv
TbTipoVinculoAlunoResponsavel.csv
Tb

In [130]:
# Dataframes importantes
alunos_turmas_df = TbAlunos['TbAlunoTurma']
alunos_df = TbAlunos['TbAluno']
alunos_turmas_hist_df = TbAlunos['TbAlunoTurmaHistorico']
disciplinas_df = TbSerie['TbSerie']
turmas_df = TbTurma['TbTurma']
situacao_aluno_turma_df = TbTurma['TbSituacaoAlunoTurma']
motivos_inat_df = TbOutras['TbMotivoInativacao']

# Target


In [131]:
target_df = pd.read_csv('/content/drive/MyDrive/Pos tech/datathon/target_df.csv')

In [132]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7313 entries, 0 to 7312
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   IdTurma                         7313 non-null   int64  
 1   IdAluno                         7313 non-null   int64  
 2   IdSituacaoAlunoTurma            7313 non-null   int64  
 3   DataSituacaoAtivo               7313 non-null   object 
 4   DataSituacaoInativo             7313 non-null   object 
 5   OrdemChamada                    7313 non-null   int64  
 6   DataHoraEfetivacaoMatricula     7313 non-null   object 
 7   IdUsuarioEfetivacaoMatricula    3166 non-null   float64
 8   ProblemaAutorizadoMatricula     2558 non-null   object 
 9   IdUsuarioAutorizacaoMatricula   2279 non-null   float64
 10  StAlunoTurmaNaoPermitePParcial  7313 non-null   bool   
 11  IdResponsavelFinanceiro         7300 non-null   float64
 12  IdTipoResponsavelFinanceiro     73

# Features

In [271]:
periodo_df = TbOutras['TbPeriodo']
alunos_df = TbAlunos['TbAluno']

In [272]:
# Criando tabela com todos os anos e mês
lst_anos = periodo_df['AnoConclusao'].unique()
lst_meses = list(range(1,13))
ano_mes_df = pd.DataFrame(list(itertools.product(lst_anos, lst_meses)), columns=['Ano', 'Mes'])
ano_mes_df['AnoMes'] = ano_mes_df['Ano'].astype(str) + ano_mes_df['Mes'].astype(str).str.zfill(2)

In [273]:
# Tabela de será usada para adicionar as features dos alunos
ano_mes_df['key'] = 1
alunos_df['key'] = 1

features_df = pd.merge(
    ano_mes_df,
    alunos_df[['key','IdAluno']].drop_duplicates(),
    on='key',
    how='left'
).drop(columns=['key'])

features_df.sort_values(by=['IdAluno','AnoMes'], inplace=True)

In [274]:
features_df

,Ano,Mes,AnoMes,IdAluno
0,2018,1,201801,3
2238,2018,2,201802,3
4476,2018,3,201803,3
6714,2018,4,201804,3
8952,2018,5,201805,3
...,...,...,...,...
179039,2024,8,202408,2262
181277,2024,9,202409,2262
183515,2024,10,202410,2262
185753,2024,11,202411,2262


# Dados de presença e faltas

In [137]:
TbDiario.keys()

dict_keys(['TbDiarioFrequencia', 'TbDiarioAluno', 'TbDiario', 'TbDiarioAula'])

In [138]:
diario_freq_df = TbDiario['TbDiarioFrequencia'] # Tabela com os frequências, uma linha por IdDiarioFrequencia
diario_df = TbDiario['TbDiario'] # Tabela com os dados do diário, uma linha por IdDiario
diario_aula_df = TbDiario['TbDiarioAula'] # Tabela com os dados da aula, uma linha por IdDiarioAula
diario_aluno_df = TbDiario['TbDiarioAluno'] # Tabela com os dados do aluno, uma linha por IdDiarioAluno

In [139]:
# Cruzandos as tabela para montar um analítico de frequências
print("Shape de diario_freq_df:", diario_freq_df.shape)
analitico_freq_df = diario_freq_df.merge(diario_aula_df, on='IdDiarioAula', how='left')
analitico_freq_df = analitico_freq_df.merge(diario_df, on='IdDiario', how='left')
analitico_freq_df = analitico_freq_df.merge(diario_aluno_df, on=['IdDiario','IdAluno'], how='left')

print("Shape de analitico_freq_df:", analitico_freq_df.shape)

Shape de diario_freq_df: (313160, 4)
Shape de analitico_freq_df: (313160, 26)


In [140]:
# Limpeza de colunas
analitico_freq_df = cleanning_cols_dates(analitico_freq_df) # Limpeza de colunas de datas

Lista de colunas de datas: ['DataAula', 'DataHoraLimiteChamadaOnline', 'DataLimiteDigitacao', 'DataBloqueioDigitacaoAula', 'DataInicial', 'DataFinal']
Essa coluna "DataHoraLimiteChamadaOnline" não pode ser convertida para data


<ipython-input-128-067541997676>:175: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_[col] = pd.to_datetime(df_[col])


In [141]:
# Adicionando colunas
analitico_freq_df['AnoMesAula'] = analitico_freq_df['DataAula'].dt.strftime('%Y%m').astype(int)

In [142]:
# Consolidando frequências por ano, mês, diário e disciplina
frequencia_aluno_turma_df = analitico_freq_df.groupby(by=[
    'IdAluno',
    'IdTurma',
    # 'IdDiario',
    'IdDisciplina',
    'AnoMesAula',
    'StPresencaFalta'
], as_index=False).agg(
    QtdAulas=('IdDiarioFrequencia', 'nunique'),
    # MaxQtdeMinimaAulas=('QtdeMinimaAulas', 'max'),
    # MinQtdeMinimaAulas=('QtdeMinimaAulas', 'min'),
    # MaxQtdeMaximaAulas=('QtdeMaximaAulas', 'max'),
    # MinQtdeMaximaAulas=('QtdeMaximaAulas', 'min')
)

# # Removendo colunas
# if frequencia_aluno_turma_df[
#     (frequencia_aluno_turma_df['MaxQtdeMinimaAulas'] != frequencia_aluno_turma_df['MinQtdeMinimaAulas']) |
#     (frequencia_aluno_turma_df['MaxQtdeMaximaAulas'] != frequencia_aluno_turma_df['MinQtdeMaximaAulas'])
# ].shape[0] == 0:
#     frequencia_aluno_turma_df = frequencia_aluno_turma_df.drop(columns=['MaxQtdeMinimaAulas','MaxQtdeMaximaAulas'])
#     frequencia_aluno_turma_df = frequencia_aluno_turma_df.rename(columns={'MinQtdeMinimaAulas': 'QtdeMinimaAulas', 'MinQtdeMaximaAulas': 'QtdeMaximaAulas'})

In [143]:
# Pivoteando a coluna de StPresencaFalta
frequencia_aluno_turma_df = frequencia_aluno_turma_df.pivot(
    index=[
        'IdAluno',
        'IdTurma',
        # 'IdDiario',
        'IdDisciplina',
        'AnoMesAula'],
    columns='StPresencaFalta',
    values=[
        'QtdAulas',
        # 'QtdeMinimaAulas',
        # 'QtdeMaximaAulas'
        ]
).reset_index()

# # Tratando colunas
# frequencia_aluno_turma_df = coalesce_multiple_columns(df=frequencia_aluno_turma_df, lst_cols_to_coalesce=[('QtdeMinimaAulas', 'P'),('QtdeMinimaAulas', 'F'),('QtdeMinimaAulas', 'J')], new_col=('QtdeMinimaAulas', ''), drop_cols=True)
# frequencia_aluno_turma_df = coalesce_multiple_columns(df=frequencia_aluno_turma_df, lst_cols_to_coalesce=[('QtdeMaximaAulas', 'P'),('QtdeMaximaAulas', 'F'),('QtdeMaximaAulas', 'J')], new_col=('QtdeMaximaAulas', ''), drop_cols=True)

frequencia_aluno_turma_df.columns = [
    'IdAluno',
    'IdTurma',
    # 'IdDiario',
    'IdDisciplina',
    'AnoMesAula',
    'QtdAulasFalta',
    'QtdAulasJustificada',
    'QtdAulasPresente',
    # 'QtdeMinimaAulas',
    # 'QtdeMaximaAulas'
]

In [144]:
# Tratando nulos
frequencia_aluno_turma_df = frequencia_aluno_turma_df.fillna({'QtdAulasFalta': 0, 'QtdAulasJustificada': 0, 'QtdAulasPresente': 0})

## Criando novas colunas
frequencia_aluno_turma_df['QtdAulasTotal'] = frequencia_aluno_turma_df['QtdAulasFalta'] + frequencia_aluno_turma_df['QtdAulasJustificada'] + frequencia_aluno_turma_df['QtdAulasPresente']

# Criando Features acumuladas
frequencia_aluno_turma_df = frequencia_aluno_turma_df.sort_values(by=['IdAluno','IdTurma','IdDisciplina','AnoMesAula'], ascending=[True,True,True,True])
frequencia_aluno_turma_df['QtdAulasFaltaAcum'] = frequencia_aluno_turma_df.groupby(by=['IdAluno','IdTurma','IdDisciplina'])['QtdAulasFalta'].cumsum()
frequencia_aluno_turma_df['QtdAulasJustificadaAcum'] = frequencia_aluno_turma_df.groupby(by=['IdAluno','IdTurma','IdDisciplina'])['QtdAulasJustificada'].cumsum()
frequencia_aluno_turma_df['QtdAulasPresenteAcum'] = frequencia_aluno_turma_df.groupby(by=['IdAluno','IdTurma','IdDisciplina'])['QtdAulasPresente'].cumsum()
frequencia_aluno_turma_df['QtdAulasTotalAcum'] = frequencia_aluno_turma_df.groupby(by=['IdAluno','IdTurma','IdDisciplina'])['QtdAulasTotal'].cumsum()

# Criando features de %
frequencia_aluno_turma_df['PercFalta'] = frequencia_aluno_turma_df['QtdAulasFalta'] / frequencia_aluno_turma_df['QtdAulasTotal']
frequencia_aluno_turma_df['PercJustificada'] = frequencia_aluno_turma_df['QtdAulasJustificada'] / frequencia_aluno_turma_df['QtdAulasTotal']
frequencia_aluno_turma_df['PercPresente'] = frequencia_aluno_turma_df['QtdAulasPresente'] / frequencia_aluno_turma_df['QtdAulasTotal']

frequencia_aluno_turma_df['PercFaltaAcum'] = frequencia_aluno_turma_df['QtdAulasFaltaAcum'] / frequencia_aluno_turma_df['QtdAulasTotalAcum']
frequencia_aluno_turma_df['PercJustificadaAcum'] = frequencia_aluno_turma_df['QtdAulasJustificadaAcum'] / frequencia_aluno_turma_df['QtdAulasTotalAcum']
frequencia_aluno_turma_df['PercPresenteAcum'] = frequencia_aluno_turma_df['QtdAulasPresenteAcum'] / frequencia_aluno_turma_df['QtdAulasTotalAcum']

In [145]:
frequencia_aluno_turma_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104754 entries, 0 to 104753
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   IdAluno                  104754 non-null  int64  
 1   IdTurma                  104754 non-null  int64  
 2   IdDisciplina             104754 non-null  int64  
 3   AnoMesAula               104754 non-null  int64  
 4   QtdAulasFalta            104754 non-null  float64
 5   QtdAulasJustificada      104754 non-null  float64
 6   QtdAulasPresente         104754 non-null  float64
 7   QtdAulasTotal            104754 non-null  float64
 8   QtdAulasFaltaAcum        104754 non-null  float64
 9   QtdAulasJustificadaAcum  104754 non-null  float64
 10  QtdAulasPresenteAcum     104754 non-null  float64
 11  QtdAulasTotalAcum        104754 non-null  float64
 12  PercFalta                104754 non-null  float64
 13  PercJustificada          104754 non-null  float64
 14  Perc

In [ ]:
# Adicionando as dados de frequencia na tabela de features
features_df = pd.merge(
    features_df,
    frequencia_aluno_turma_df,
    on=['IdAluno','AnoMesAula'],
    how='left'
)

# Dados de notas

In [255]:
fase_nota_aluno_df = TbFase['TbFaseNotaAluno']
fase_nota_df = TbFase['TbFaseNota']
fase_nota_disciplina_turma_df = TbFase['TbFaseNotaDisciplinaTurma']
fase_nota_origem_destino_df = TbFase['TbFaseNotaOrigemDestino']
meta_fase_nota_df = TbMeta['TbMetaFaseNota']
periodo_df = TbOutras['TbPeriodo']
serie_df = TbSerie['TbSerie']
historico_df = TbHistorico['TbHistorico']
historico_notas_df = TbHistorico['TbHistoricoNotas']

In [256]:
TbFase.keys()

dict_keys(['TbFaseNotaAluno', 'TbFaseNota', 'TbFaseNotaDisciplinaTurma', 'TbFaseNotaOrigemDestino'])

In [257]:
# Consolidando Data de Inicio e Fim de cada IdFaseNota
print("shape meta_fase_nota_df:", meta_fase_nota_df.shape)

# infos_tempo_fase_nota_df = pd.merge(
#     meta_fase_nota_df,
#     periodo_df[['IdPeriodo','DataInicial','DataFinal']].rename(
#         columns={'DataInicial': 'DataInicialPeriodoAula', 'DataFinal': 'DataFinalPeriodoAula'}
#     ),
#     on='IdPeriodo',
#     how='left'
# )

fase_nota_df = pd.merge(
    fase_nota_df,
    periodo_df[['IdPeriodo','DataInicial','DataFinal']].rename(
        columns={'DataInicial': 'DataInicialPeriodo', 'DataFinal': 'DataFinalPeriodo'}
    ),
    on='IdPeriodo',
    how='left'
)

print("shape fase_nota_df:", fase_nota_df.shape)

# Criando datas auxiliares
fase_nota_df = cleanning_cols_dates(fase_nota_df)

# Criando colunas
fase_nota_df['AnoPeriodoAula'] = fase_nota_df['DataInicialPeriodo'].dt.strftime('%Y').astype(int)
fase_nota_df['DataInicialFaseNota'] = np.select(
    [
        (fase_nota_df['DataInicialPeriodoAula'].notnull()).astype('bool'),
        ((fase_nota_df['DataInicialPeriodoAula'].isnull()) & (fase_nota_df['NomeFase'] == '1º SEMESTRE')).astype('bool'),
        ((fase_nota_df['DataInicialPeriodoAula'].isnull()) & (fase_nota_df['NomeFase'] == '2º SEMESTRE')).astype('bool')
    ],
    [
        fase_nota_df['DataInicialPeriodoAula'],
        fase_nota_df['DataInicialPeriodo'],
        pd.to_datetime(fase_nota_df['AnoPeriodoAula'].astype('str') + '-07-25')
    ],
    default=None
)

fase_nota_df['DataFinalFaseNota'] = np.select(
    [
        (fase_nota_df['DataFinalPeriodoAula'].notnull()).astype('bool'),
        ((fase_nota_df['DataFinalPeriodoAula'].isnull()) & (fase_nota_df['NomeFase'] == '1º SEMESTRE')).astype('bool'),
        ((fase_nota_df['DataFinalPeriodoAula'].isnull()) & (fase_nota_df['NomeFase'] == '2º SEMESTRE')).astype('bool')
    ],
    [
        fase_nota_df['DataFinalPeriodoAula'],
        pd.to_datetime(fase_nota_df['AnoPeriodoAula'].astype('str') + '-07-10'),
        fase_nota_df['DataFinalPeriodo']
    ],
    default=None
)


# Consolidando analítico notas das fase por aluno
print("shape fase_nota_aluno_df:", fase_nota_aluno_df.shape)

analitico_fase_nota_aluno_df = pd.merge(
    fase_nota_aluno_df,
    fase_nota_df,
    on='IdFaseNota',
    how='left'
)

print("shape analitico_fase_nota_aluno_df:", analitico_fase_nota_aluno_df.shape)


shape meta_fase_nota_df: (162, 5)
shape fase_nota_df: (1092, 40)
Lista de colunas de datas: ['DataInicioExibicao', 'DataInicialPeriodoAula', 'DataFinalPeriodoAula', 'DataInicialPeriodo', 'DataFinalPeriodo']
shape fase_nota_aluno_df: (121656, 20)
shape analitico_fase_nota_aluno_df: (121656, 62)


In [215]:
# Consolidando qtd de séries aprovadas
qtd_series_aprovadas_por_ano_df = historico_df.groupby(by=['IdAluno','AnoConclusao','ResultadoFinal'], as_index=False).agg(QtdSeries=('CodigoSerie','nunique'))
qtd_series_aprovadas_por_ano_df = qtd_series_aprovadas_por_ano_df.pivot(index=['IdAluno','AnoConclusao'], columns='ResultadoFinal', values='QtdSeries').reset_index()
qtd_series_aprovadas_por_ano_df.columns = ['IdAluno','AnoConclusao','QtdSeriesA','QtdSeriesC']
qtd_series_aprovadas_por_ano_df = qtd_series_aprovadas_por_ano_df.fillna({'QtdSeriesA': 0, 'QtdSeriesC': 0})
qtd_series_aprovadas_por_ano_df = qtd_series_aprovadas_por_ano_df.sort_values(by=['IdAluno','AnoConclusao'], ascending=[True,True])

In [241]:
# Consolidando dados de histórico
print("shape historico_notas_df:", historico_notas_df.shape)

analitico_historico_df = pd.merge(
    historico_notas_df,
    serie_df[['IdSerie','NomeSerie','IdCurso','IdProximaSerie','IdServicoMensalidade','CodigoSerie']],
    on='CodigoSerie',
    how='left'
)

analitico_historico_df['NotaFinal'] = pd.to_numeric(analitico_historico_df['NotaFinal'].str.replace(',','.').str.strip(), errors='coerce')


nota_media_aluno_por_disciplina_df = analitico_historico_df.groupby(by=['IdAluno','AnoConclusao','IdDisciplina'], as_index=False).agg(NotaMedia=('NotaFinal','mean'))

print("shape analitico_historico_df:", analitico_historico_df.shape)

shape historico_notas_df: (60, 11)
shape analitico_historico_df: (60, 16)
